In [ ]:
!pip install dagshub mlflow neuralforecast --quiet

import warnings
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore")

print("Done!")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import mlflow.sklearn
from datetime import datetime
import joblib
import dagshub
import mlflow
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from tqdm import tqdm
from sklearn.compose import ColumnTransformer
from statsmodels.tsa.arima.model import ARIMA
import os
from neuralforecast.models import DLinear, PatchTST
from neuralforecast import NeuralForecast

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

In [ ]:
# class definitions

class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["week"] = (
            X["Date"].dt.to_period("W")
            .rank(method="dense")
            .astype(int) - 1
        )
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)
        X = X.drop(columns=["Date"])
        return X

date_features = ["week", "sin_13", "cos_13", "sin_23", "cos_23"]

class LagFeatureAdder:
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.sort_values(["Store", "Dept", "Date"])
        df["lag_1"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
        df["lag_52"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)
        return df

lag_features = ["lag_1", "lag_52"]

added_features = date_features + lag_features

class ColumnTransformerWithNames(ColumnTransformer):
    def get_feature_names_out(self, input_features=None):
        return super().get_feature_names_out(input_features)

    def transform(self, X):
        X_transformed = super().transform(X)
        # Get feature names for columns
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        # print("with name transform - ", type(res))
        return res

    def fit_transform(self, X, y=None):
        X_transformed = super().fit_transform(X, y)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        # print("with name fit_transform - ", type(res))
        return res

class MultiIndexKeeper(BaseEstimator, TransformerMixin):
    def __init__(self, index_cols=["Date", "Store", "Dept"]):
        self.index_cols = index_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X.set_index(self.index_cols, drop=False, inplace=True)
        return X

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        return X.drop(columns=self.columns, errors="ignore")

def extract_onehot_value(row, prefix, default=1):
    for col in row.index:
        if col.startswith(prefix) and row[col] == 1:
            return int(col.split("_")[1])
    return default

class ResidualRegressor(BaseEstimator, RegressorMixin):
    def extract_onehot_value(self, row, prefix, default=1):
        for col in row.index:
            if col.startswith(prefix) and row[col] == 1:
                return int(col.split("_")[1])
        return default
    
    def predict(self, X):
        pred_lookup = {}
        weekly_preds = pd.Series(index=X.index, dtype=float)
        
        for week in tqdm(X["week"].sort_values().unique(), desc="Recursive prediction"):
            day_rows = X[X["week"] == week].copy()
        
            for idx, row in day_rows.iterrows():
                store = extract_onehot_value(row, "Store_")
                dept = extract_onehot_value(row, "Dept_")
        
                # Get keys for previous lags
                key_1 = (store, dept, week - 1)
                key_52 = (store, dept, week - 52)
        
                lag_1 = pred_lookup.get(key_1, row["lag_1"])
                lag_52 = pred_lookup.get(key_52, row["lag_52"])
        
                day_rows.at[idx, "lag_1"] = lag_1
                day_rows.at[idx, "lag_52"] = lag_52
        
            # Predict all rows for this day in one batch
            y_preds = self.pred_f(day_rows)
        
            # Assign predictions back
            weekly_preds[day_rows.index] = y_preds
            
            # Update lookup for future lag access
            for idx, pred in zip(day_rows.index, y_preds):
                row = day_rows.loc[idx]
                
                store = extract_onehot_value(row, "Store_")
                dept = extract_onehot_value(row, "Dept_")
                
                key = (store, dept, row["week"])
                pred_lookup[key] = pred
    
        return weekly_preds.fillna(0).to_numpy()

    def pred_f(self, X):
        return self.base_model_.predict(X) + self.residual_model_.predict(X)

class ARIMARegressor(BaseEstimator, RegressorMixin):
    def predict(self, X):
        if not isinstance(X.index, pd.MultiIndex):
            raise ValueError("X must have a MultiIndex")
    
        preds = pd.Series(index=X.index, dtype=float)
    
        # Group X by store-dept pair (based on index levels)
        grouped = X.groupby(level=[self.store_level, self.dept_level])
    
        for (store, dept), group_df in grouped:
            if dept == 1:
                print("store: ", store)
            model = self.models_.get((store, dept))
            if model is None:
                preds.loc[group_df.index] = self.avgs_.get((store, dept), 0)
                continue

            # exog = group_df.copy()
    
            # Forecast N steps = number of rows in this group
            forecast = model.forecast(steps=len(group_df))
            preds.loc[group_df.index] = forecast.to_numpy()
    
        return preds.to_numpy()

class DLinearRegressor(BaseEstimator, RegressorMixin):
    def predict(self, X):
        df = X.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        forecast_df = self.nf_.predict()
        forecast_df = forecast_df.rename(columns={"DLinear": "y_hat"})

        merged = df.merge(forecast_df, on=["unique_id", "ds"], how="left")
        preds = pd.Series(data=merged["y_hat"].fillna(0).values, index=X.index)

        return preds.to_numpy()

class PatchTSTRegressor(BaseEstimator, RegressorMixin):
    def predict(self, X):
        df = X.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        forecast_df = self.nf_.predict()
        forecast_df = forecast_df.rename(columns={"PatchTST": "y_hat"})

        merged = df.merge(forecast_df, on=["unique_id", "ds"], how="left")
        preds = pd.Series(data=merged["y_hat"].fillna(0).values, index=X.index)

        return preds.to_numpy()

class NBEATSRegressor(BaseEstimator, RegressorMixin):
    def predict(self, X):
        df = X.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        forecast_df = self.nf_.predict()
        forecast_df = forecast_df.rename(columns={"NBEATS": "y_hat"})

        merged = df.merge(forecast_df, on=["unique_id", "ds"], how="left")
        preds = pd.Series(data=merged["y_hat"].fillna(0).values, index=X.index)

        return preds.to_numpy()

class TFTRegressor(BaseEstimator, RegressorMixin):
    def predict(self, X):
        df = X.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        forecast_df = self.nf_.predict()
        forecast_df = forecast_df.rename(columns={"TFT": "y_hat"})

        merged = df.merge(forecast_df, on=["unique_id", "ds"], how="left")
        preds = pd.Series(data=merged["y_hat"].fillna(0).values, index=X.index)

        return preds.to_numpy()

In [ ]:
local_path = mlflow.artifacts.download_artifacts(
    artifact_uri="mlflow-artifacts:/4e741da8978446259b80b5b149310c3c/ad8aa507a9334013997c9f09376cfa21/artifacts/model.pkl"
)

print("Downloaded file size:", os.path.getsize(local_path) / (1024 ** 2), "MB")

model = joblib.load(local_path)

print("Done!")

In [ ]:
# load and add lag features

test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip", parse_dates=["Date"])
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = test.merge(features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(stores, on="Store", how="left")

def add_lag_features(df):
    train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
    train = train[["Store", "Dept", "Date", "Weekly_Sales"]]
    train = train.sort_values(["Store", "Dept", "Date"])
    
    full = pd.concat([train, df], axis=0)
    full = full.sort_values(["Store", "Dept", "Date"])
    
    full["lag_1"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
    full["lag_52"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)

    res = full[full["Weekly_Sales"].isna()].copy()

    return res

df = add_lag_features(df)

X_test = df.drop(columns=["Weekly_Sales"], errors="ignore")

print("Done!")

In [ ]:
preds = model.predict(X_test)

submission = pd.DataFrame()
submission["Weekly_Sales"] = preds
submission["Id"] = X_test["Store"].astype(str) + "_" + X_test["Dept"].astype(str) + "_" + X_test["Date"].dt.strftime("%Y-%m-%d")
# print(submission)
submission.to_csv("submission.csv", index=False)
print("✅ Submission saved as submission.csv")